In [1]:
!pip install spotipy

In [3]:
!pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.4-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached numpy-2.2.4-cp312-cp312-win_amd64.whl (12.6 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)


In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [ ]:
client_credentials_manager = SpotifyClientCredentials(
    client_id="####",
    client_secret="####",
)

In [ ]:
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [16]:
# Playlist link
playlist_link = "https://open.spotify.com/playlist/3cEYpjA9oz9GiPac4AsH4n"
# Extract playlist URI
playlist_URI = playlist_link.split("/")[-1].split("?")[0]

# Fetch playlist tracks
data = sp.playlist_tracks(playlist_URI)

In [ ]:
def album(data):
    album_list = []

    for row in data["items"]:
        album_id = row["track"]["album"]["id"]
        album_name = row["track"]["album"]["name"]
        album_release_date = row["track"]["album"]["release_date"]
        album_total_tracks = row["track"]["album"]["total_tracks"]
        album_url = row["track"]["album"]["external_urls"]["spotify"]

        album_element = {
            "album_id": album_id,
            "name": album_name,
            "release_date": album_release_date,
            "total_tracks": album_total_tracks,
            "url": album_url,
        }
        album_list.append(album_element)
    return album_list

In [ ]:
def artist(data):
    artist_list = []

    for row in data["items"]:
        for artist in row["track"]["artists"]:
            artist_dict = {
                "artist_id": artist["id"],
                "artist_name": artist["name"],
                "external_url": artist["href"],
            }
            artist_list.append(artist_dict)

    return artist_list

In [ ]:
def songs(data):
    song_list = []
    for row in data["items"]:
        song_id = row["track"]["id"]
        song_name = row["track"]["name"]
        song_duration = row["track"]["duration_ms"]
        song_url = row["track"]["external_urls"]["spotify"]
        song_popularity = row["track"]["popularity"]
        song_added = row["added_at"]
        album_id = row["track"]["album"]["id"]
        artist_id = row["track"]["album"]["artists"][0]["id"]
        song_element = {
            "song_id": song_id,
            "song_name": song_name,
            "duration_ms": song_duration,
            "url": song_url,
            "popularity": song_popularity,
            "song_added": song_added,
            "album_id": album_id,
            "artist_id": artist_id,
        }
        song_list.append(song_element)

    return song_list

In [21]:
album_df = pd.DataFrame.from_dict(album(data))
artist_df = pd.DataFrame.from_dict(artist(data))
song_df = pd.DataFrame.from_dict(songs(data))

In [ ]:
album_df = album_df.drop_duplicates(subset=["album_id"])
artist_df = artist_df.drop_duplicates(subset=["artist_id"])
song_df = song_df.drop_duplicates(subset=["song_id"])

In [ ]:
album_df["release_date"] = pd.to_datetime(album_df["release_date"])
song_df["song_added"] = pd.to_datetime(song_df["song_added"])

In [24]:
album_df.head(10)

,album_id,name,release_date,total_tracks,url
0,2pANdqPvxInB0YvcDiw4ko,Progressive Psy Trance Picks Vol.8,2012-04-02,20,https://open.spotify.com/album/2pANdqPvxInB0Yv...
1,6nlfkk5GoXRL1nktlATNsy,Wellness & Dreaming Source,2015-01-09,25,https://open.spotify.com/album/6nlfkk5GoXRL1nk...
2,4hnqM0JK4CM1phwfq1Ldyz,This Is Happening,2010-05-17,9,https://open.spotify.com/album/4hnqM0JK4CM1phw...
3,2usKFntxa98WHMcyW6xJBz,Glenn Horiuchi Trio / Gelenn Horiuchi Quartet:...,2011-04-01,8,https://open.spotify.com/album/2usKFntxa98WHMc...
4,0ivM6kSawaug0j3tZVusG2,All The Best (Spanish Version),2007-01-01,18,https://open.spotify.com/album/0ivM6kSawaug0j3...


In [25]:
artist_df.head(10)

,artist_id,artist_name,external_url
0,6eSdhw46riw2OUHgMwR8B5,Odiseo,https://api.spotify.com/v1/artists/6eSdhw46riw...
1,5VQE4WOzPu9h3HnGLuBoA6,Vlasta Marek,https://api.spotify.com/v1/artists/5VQE4WOzPu9...
2,066X20Nz7iquqkkCW6Jxy6,LCD Soundsystem,https://api.spotify.com/v1/artists/066X20Nz7iq...
3,272ArH9SUAlslQqsSgPJA2,Glenn Horiuchi Trio,https://api.spotify.com/v1/artists/272ArH9SUAl...
4,2KftmGt9sk1yLjsAoloC3M,Zucchero,https://api.spotify.com/v1/artists/2KftmGt9sk1...


In [26]:
song_df.head(10)

,song_id,song_name,duration_ms,url,popularity,song_added,album_id,artist_id
0,4rzfv0JLZfVhOhbSQ8o5jZ,Api,376000,https://open.spotify.com/track/4rzfv0JLZfVhOhb...,1,2015-01-15 12:39:22+00:00,2pANdqPvxInB0YvcDiw4ko,0LyfQWJT6nXafLPZqxe9Of
1,5o3jMYOSbaVz3tkgwhELSV,Is,730066,https://open.spotify.com/track/5o3jMYOSbaVz3tk...,0,2015-01-15 12:40:03+00:00,6nlfkk5GoXRL1nktlATNsy,0LyfQWJT6nXafLPZqxe9Of
2,4Cy0NHJ8Gh0xMdwyM9RkQm,All I Want,401440,https://open.spotify.com/track/4Cy0NHJ8Gh0xMdw...,48,2015-01-15 12:22:30+00:00,4hnqM0JK4CM1phwfq1Ldyz,066X20Nz7iquqkkCW6Jxy6
3,6hvFrZNocdt2FcKGCSY5NI,Endpoints,358760,https://open.spotify.com/track/6hvFrZNocdt2FcK...,0,2015-01-15 12:40:35+00:00,2usKFntxa98WHMcyW6xJBz,272ArH9SUAlslQqsSgPJA2
4,2E2znCPaS8anQe21GLxcvJ,You Are So Beautiful,176093,https://open.spotify.com/track/2E2znCPaS8anQe2...,0,2015-01-15 12:41:10+00:00,0ivM6kSawaug0j3tZVusG2,2KftmGt9sk1yLjsAoloC3M
